In [121]:
import os
os.environ["OMP_NUM_THREADS"]="1"
os.environ["MKL_NUM_THREADS"]="1"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import sys
# PDC-Net path
sys.path.append('../DenseMatching/')
from easydict import EasyDict as easydic
from utils_flow.pixel_wise_mapping import remap_using_flow_fields
from model_selection import select_model

from skimage import io
from matplotlib import pyplot as plt
import cv2
import numpy as np
import torch
import torch.nn.functional as F
import warnings

# LIFE path
sys.path.append('./LIFE2')
sys.path.append('./LIFE2/core')
# sys.path.append('./RAFT')
# sys.path.append('./RAFT/core')
# sys.path.append('./Twins')
# sys.path.append('./Twins/core')
# sys.path.append('./Twins-new')
# sys.path.append('./Twins-new/core')
# sys.path.append('./Occ')
# sys.path.append('./Occ/core')
# sys.path.append('./MultiScale')
# sys.path.append('./MultiScale/core')
from flow_estimator import Flow_estimator
if './Twins-new' in sys.path:
    from core.utils.forward_warp import ForwardWarp
# from core.utils.utils import image_flow_warp
# from core.utils.utils import coords_grid
from config import get_life_args, get_raft_args, get_twins_args

# SPSG path
sys.path.append('./SPSG')
sys.path.append('./SPSG/models')
from pathlib import Path 
import random 
import matplotlib.cm as cm 
from SPSG.models.matching import Matching
from SPSG.models.utils import (compute_pose_error, compute_epipolar_error,
                          estimate_pose, make_matching_plot,
                          error_colormap, AverageTimer, pose_auc, read_image,
                          rotate_intrinsics, rotate_pose_inplane,
                          scale_intrinsics, frame2tensor, plot_image_pair,
                          plot_keypoints, plot_matches) 
torch.set_grad_enabled(False)

# metrics
from skimage.metrics import mean_squared_error as mse 
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import datetime
# psnr
from skimage.util.dtype import dtype_range
from skimage._shared.utils import warn, check_shape_equality
# ssim
from skimage.util.arraycrop import crop
from scipy.ndimage import uniform_filter, gaussian_filter

%matplotlib inline


In [122]:
from PIL import Image, ImageColor
import plotly.graph_objects as go
import plotly.colors

def get_color(colorscale_name, loc):
    from _plotly_utils.basevalidators import ColorscaleValidator
    # first parameter: Name of the property being validated
    # second parameter: a string, doesn't really matter in our use case
    cv = ColorscaleValidator("colorscale", "")
    # colorscale will be a list of lists: [[loc1, "rgb1"], [loc2, "rgb2"], ...]
    colorscale = cv.validate_coerce(colorscale_name)

    if hasattr(loc, "__iter__"):
        intermediate_colors = [get_continuous_color(colorscale, x) for x in loc]
        return intermediate_colors
    return get_continuous_color(colorscale, loc)


def get_continuous_color(colorscale, intermed):
    """
    Plotly continuous colorscales assign colors to the range [0, 1]. This function computes the intermediate
    color for any value in that range.
    Plotly doesn't make the colorscales directly accessible in a common format.
    Some are ready to use:
        colorscale = plotly.colors.PLOTLY_SCALES["Greens"]
    Others are just swatches that need to be constructed into a colorscale:
        viridis_colors, scale = plotly.colors.convert_colors_to_same_type(plotly.colors.sequential.Viridis)
        colorscale = plotly.colors.make_colorscale(viridis_colors, scale=scale)
    :param colorscale: A plotly continuous colorscale defined with RGB string colors.
    :param intermed: value in the range [0, 1]
    :return: color in rgb string format
    :rtype: str
    """
    if len(colorscale) < 1:
        raise ValueError("colorscale must have at least one color")

    hex_to_rgb = lambda c: "rgb" + str(ImageColor.getcolor(c, "RGB"))

    if intermed <= 0 or len(colorscale) == 1:
        c = colorscale[0][1]
        return c if c[0] != "#" else hex_to_rgb(c)
    if intermed >= 1:
        c = colorscale[-1][1]
        return c if c[0] != "#" else hex_to_rgb(c)

    for cutoff, color in colorscale:
        if intermed > cutoff:
            low_cutoff, low_color = cutoff, color
        else:
            high_cutoff, high_color = cutoff, color
            break

    if (low_color[0] == "#") or (high_color[0] == "#"):
        # some color scale names (such as cividis) returns:
        # [[loc1, "hex1"], [loc2, "hex2"], ...]
        low_color = hex_to_rgb(low_color)
        high_color = hex_to_rgb(high_color)

    intermediate_color = plotly.colors.find_intermediate_color(
        lowcolor=low_color,
        highcolor=high_color,
        intermed=((intermed - low_cutoff) / (high_cutoff - low_cutoff)),
        colortype="rgb",
    )
    return intermediate_color

def get_plotly_colors(num_points, colorscale):
    color_steps = torch.linspace(start=0, end=1, steps=num_points).tolist()
    colors = get_color(colorscale, color_steps)
    colors = [plotly.colors.unlabel_rgb(color) for color in colors]
    colors = torch.tensor(colors, dtype=torch.float, device='cuda').view(1, num_points, 3)
    colors = colors.div(255.0).add(-0.5).mul(2)  # Map [0, 255] RGB colors to [-1, 1]
    return colors  # (1, P, 3)

def get_colormap(flow, H, W):
    points = flow.permute(0, 2, 3, 1)
    points = points.reshape(points.size(0), points.size(1) * points.size(2), 2)  # (N, K*P, 2)
    num_points = points.size(1)
    colorscale = ['plasma']
    colors = torch.cat([get_plotly_colors(num_points, c) for c in colorscale], 1)  # (1, K*P, 3)
    colors = colors.reshape(H, W, 3).cpu().numpy() * 255 

    return colors

In [123]:
def show_input_images(scene_images, target_images):
    plt.figure(figsize=(15,15))
    num = len(scene_images)
    plt.subplot(221), plt.imshow(scene_images[0]), plt.axis('off')
    plt.subplot(222), plt.imshow(target_images[0]), plt.axis('off')
    plt.subplot(223), plt.imshow(scene_images[num-1]), plt.axis('off')
    plt.subplot(224), plt.imshow(target_images[num-1]), plt.axis('off')
    # plt.show()

In [124]:
def blend(out, source, scene, blend_type, mask=None):
    if mask is None:
        intensity = np.linalg.norm(out, axis=2)
        mask = (intensity == 0)[:,:,np.newaxis]   
    else:
        mask = mask
        
    if blend_type == 'light':
        result = (out * (1 - mask) + source * mask).astype(np.uint8)
    else:
        result = (out * (1 - mask) + scene * mask).astype(np.uint8) 
    return result, mask


In [125]:
def blend_homography(scene_images, target, source=None, blend_type='scale', detector='SIFT', use_colormap=True):
    #H, W = 480, 640
    #target = cv2.resize(target, (W, H)) 
    if detector == 'ORB':
        detect = cv2.ORB_create()    
    elif detector == 'SIFT':
        detect = cv2.SIFT_create()
    else:
        raise ValueError('detector not implemented')
    blends = []
    masks = []
    for scene in scene_images:
        #ori_H, ori_W = scene.shape[:2]
        #scene = cv2.resize(scene, (W, H))        
        kp1, des1 = detect.detectAndCompute(target, None)
        kp2, des2 = detect.detectAndCompute(scene, None)           
        if des1 is None or des2 is None:
            blends.append(None)   
            continue
        
        if detector == 'ORB':
            bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)                
            matches = bf.match(des1, des2)
            
        elif detector == 'SIFT':
            FLANN_INDEX_KDTREE = 0
            indexParams = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
            searchParams = dict(checks=50)
            flann = cv2.FlannBasedMatcher(indexParams, searchParams)    
            if len(des1)<2 or len(des2)<2:
                blends.append(None)
                continue
            matches = flann.knnMatch(des1, des2, k=2)
            matches = [m for m,n in matches if m.distance < 0.7*n.distance]
        else:
            raise ValueError('detector {} not implemented'.format(detector))
        if len(matches) < 4:
            blends.append(None)
            continue        
        matches = sorted(matches, key=lambda x: x.distance)
        #print(len(matches))
        #img3 = cv2.drawMatches(target, kp1, scene, kp2, matches[:50], target, flags=2)
        src_pts = np.float32([kp1[m.queryIdx].pt for m in matches[:50]]).reshape(-1,1,2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches[:50]]).reshape(-1,1,2)        

        M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        if M is None:
            blends.append(None)
            continue
        out = cv2.warpPerspective(target, M, (scene.shape[1], scene.shape[0]))        

        blend_i, mask_i = blend(out, source, scene, blend_type)
        #blend_i = cv2.resize(blend_i, (ori_W, ori_H))
        blends.append(blend_i)
        masks.append(mask_i)
    return blends, masks
    plt.figure(figsize=(20,30))
    plt.subplot(131), plt.imshow(target[:,:,::-1]), plt.title('target')
    plt.subplot(132), plt.imshow(scene[:,:,::-1]), plt.title('scene')
    plt.subplot(133), plt.imshow(blend[:,:,::-1]), plt.title('output')

In [126]:
def blend_pdc(scene_images, target, flow_estimator, estimate_uncertainty, source=None, blend_type='scale', use_colormap=True):            
    #H, W = 480, 640
    #img1 = cv2.resize(target, (W, H))    
    blends = []
    masks = []
    for scene in scene_images:        
        #H_ori, W_ori = scene.shape[:2]                
        #img2 = cv2.resize(scene, (W, H))
        target = cv2.resize(target, (scene.shape[1], scene.shape[0]))
        Is_original = np.ascontiguousarray(target)
        It_original = np.ascontiguousarray(scene)
        Is_tensor = torch.from_numpy(Is_original).permute(2,0,1).unsqueeze(0)
        It_tensor = torch.from_numpy(It_original).permute(2,0,1).unsqueeze(0)
        with torch.no_grad():
            flow_est, uncertainty_est = flow_estimator.estimate_flow_and_confidence_map(Is_tensor, It_tensor)

        flow = flow_est[0].permute([1,2,0]).cpu().detach().numpy()
        out = remap_using_flow_fields(Is_original, flow[:,:,0], flow[:,:,1])
        #img_warp = cv2.resize(img_warp, (W_ori, H_ori))
        
        blend_i, mask_i = blend(out, source, scene, blend_type) 
        #blend_i = cv2.resize(blend_i, (W_ori, H_ori))
        blends.append(blend_i)
        masks.append(mask_i)
    return blends, masks
        

    '''
        confidence_map = uncertainty_est['p_r'].squeeze().detach().cpu().numpy()

        plt.figure(figsize=(20,30))
        plt.subplot(141), plt.imshow(img1_ori[:,:,::-1]), plt.axis('off')
        plt.subplot(142), plt.imshow(img2_ori[:,:,::-1]), plt.axis('off')
        plt.subplot(143), plt.imshow(blend[:,:,::-1]), plt.axis('off')
        #plt.subplot(144), plt.imshow(confidence_map)
    '''

In [127]:
def PSNR(image_true, image_test, mask, heatmap=False, with_mask=True):
    check_shape_equality(image_true, image_test)    

    if image_true.dtype != image_test.dtype:
        warn("Inputs have mismatched dtype. Setting data_range based on "
                "im_true.", stacklevel=2)
    dmin, dmax = dtype_range[image_true.dtype.type]
    true_min, true_max = np.min(image_true), np.max(image_true)
    if true_max > dmax or true_min < dmin:
        raise ValueError(
            "im_true has intensity values outside the range expected for "
            "its data type. Please manually specify the data_range")
    if true_min >= 0:
        # most common case (255 for uint8, 1 for float)
        data_range = dmax
    else:
        data_range = dmax - dmin
    
    image_true = image_true.astype(np.float64)
    image_test = image_test.astype(np.float64)
    if not with_mask:                      
        error_mask = ((image_true - image_test) ** 2).astype(np.float64)
        err = np.mean(error_mask, dtype=np.float64)
    else:
        cnt = np.count_nonzero(1-mask) * image_true.shape[2]
        error_mask = ((image_true*(1-mask) - image_test*(1-mask))**2).astype(np.float64)
        sum = np.sum(error_mask)    
        err = sum/cnt
        
        # plt.figure(facecolor='white')
        # plt.subplot(121), plt.imshow(image_true.astype(np.uint8)*(1-mask))
        # plt.subplot(122), plt.imshow(image_test.astype(np.uint8)*(1-mask))
        
    score = 10 * np.log10((data_range ** 2) / err)
    if heatmap:             
        # error_heatmap = np.zeros_like(error_mask)
        error_mask = np.mean(error_mask, axis=2)   
        # np.save('./error_mask.npy', error_mask)

        x,y = np.nonzero(error_mask)
        error_mask[x,y] = 10 * np.log10((data_range ** 2) / error_mask[x,y])        
        error_mask = error_mask / 30
        # colorscale=['plasma']
        # colors = torch.cat([get_plotly_colors(600, c) for c in colorscale], 1)[0].cpu().numpy()
        # #(600, 3) (640, 480, 3) (640, 480)
        
        # for y in range(error_mask.shape[0]):
        #     for x in range(error_mask.shape[1]):
        #         print(error_mask[y,x])
        #         error_heatmap[y,x,:] = colors[error_mask[y,x].astype(np.uint8)+350,:]
        # error_heatmap = (error_heatmap * 255).astype(np.uint8)*(1-mask)
        
        error_mask = (error_mask * 255).astype(np.uint8)
        error_heatmap = cv2.applyColorMap(error_mask, cv2.COLORMAP_JET)*(1-mask)
        return score, error_heatmap
    else:
        return score

In [128]:
from scipy.ndimage import uniform_filter
def SSIM(im1, im2, mask, multichannel=True, heatmap=False, with_mask=True):
    check_shape_equality(im1, im2)

    if multichannel:
        # loop over channels
        nch = im1.shape[-1]
        mssim = np.empty(nch)
        S = np.empty(im1.shape)
        for ch in range(nch):
            ch_result = SSIM(im1[..., ch], im2[..., ch], multichannel=False, mask=mask, heatmap=heatmap)
            mssim[..., ch], S[...,ch] = ch_result
        mssim = mssim.mean()
        if heatmap:
            S = np.mean(S, axis=2)                        
            S = (S+1)/2
            S = (S * 255).astype(np.uint8)            
            error_heatmap = cv2.applyColorMap(S, cv2.COLORMAP_JET)*(1-mask)
            return mssim, error_heatmap
        else:
            return mssim

    if im1.dtype != im2.dtype:
        warn("Inputs have mismatched dtype.  Setting data_range based on im1.dtype.", stacklevel=2)
    dmin, dmax = dtype_range[im1.dtype.type]
    data_range = dmax - dmin
    
    K1 = 0.01
    K2 = 0.03
    R = data_range
    C1 = (K1 * R) ** 2
    C2 = (K2 * R) ** 2  
    S = np.ones_like(im1)
    win_size=3

    # ndimage filters need floating point data
    if not with_mask:
        im1 = im1.astype(np.float64)
        im2 = im2.astype(np.float64)
    else:
        mask = mask.squeeze()
        im1 = (im1*(1-mask)).astype(np.float64)
        im2 = (im2*(1-mask)).astype(np.float64)
    
    if heatmap:
        ux = uniform_filter(im1, size=win_size)
        uy = uniform_filter(im2, size=win_size)

        uxx = uniform_filter(im1*im1, size=win_size)
        uyy = uniform_filter(im2*im2, size=win_size)
        uxy = uniform_filter(im1*im2, size=win_size)
        vx = uxx - ux*ux 
        vy = uyy - uy*uy
        vxy = uxy - ux*uy

        A1, A2, B1, B2 = ((2 * ux * uy + C1,
                           2 * vxy + C2,
                           ux ** 2 + uy ** 2 + C1,
                           vx + vy + C2))
        D = B1 * B2
        S = (A1 * A2) / D    
    

    if not with_mask:
        mask = np.zeros_like(im1)
        x,y = np.nonzero(1-mask)
    else:
        #x,y,_ = np.nonzero(1-mask)
        x,y = np.nonzero(1-mask)
    im1_pixel = im1[x,y]
    im2_pixel = im2[x,y]

    ux = np.mean(im1_pixel)
    uy = np.mean(im2_pixel)
    uxy = np.mean(im1_pixel*im2_pixel)
    vx = np.var(im1_pixel)
    vy = np.var(im2_pixel)
    vxy = uxy - ux*uy

    ssim = (2*ux*uy+C1)*(2*vxy+C2)/((ux**2+uy**2+C1)*(vx+vy+C2))
    return ssim, S


In [129]:
def metrics(id, output_images, masks, save_root, scene_images=None, source=None, blend_type='scale', with_mask=False, save=True, heatmap=False):   
    if blend_type != 'light' and len(output_images)!=len(scene_images):
        raise ValueError('output images should have the same number as scene images')
    if save:
        save_file = os.path.join(save_root, 'output.txt')
    result = {'p':[], 's':[], 'ce':[], 'fail':0, 'psnr_heatmap':[], 'ssim_heatmap':[]}    
    if save:
        with open(save_file, 'a') as f:
            f.write('id: {}\n'.format(str(id).zfill(4)))

    for i, output in enumerate(output_images):
        if output is None: 
            result['p'].append(0)
            result['s'].append(-1)
            result['ce'].append(-1)
            result['fail'] = result['fail']+1            
            if save:
                with open(save_file, 'a') as f:
                    f.write('no.{:2d}\t Failed\n'.format(i))
            continue

        if blend_type == 'light':
            scene = source
        else:
            scene = scene_images[i]
        
        mask = masks[i]                    
        if heatmap:
            p, psnr_heatmap = PSNR(output, scene, mask=mask, heatmap=heatmap, with_mask=with_mask)
            s, ssim_heatmap = SSIM(output, scene, multichannel=True, mask=mask, heatmap=heatmap, with_mask=with_mask)
            result['psnr_heatmap'].append(psnr_heatmap)
            result['ssim_heatmap'].append(ssim_heatmap)
        else:
            p = PSNR(output, scene, mask=mask, heatmap=heatmap, with_mask=with_mask)
            s = SSIM(output, scene, multichannel=True, mask=mask, heatmap=heatmap, with_mask=with_mask)                                
        # plt.figure(facecolor='white')
        # plt.subplot(121), plt.imshow(output*(1-mask)), plt.title('output'), plt.axis('off')
        # plt.subplot(122), plt.imshow(scene*(1-mask)), plt.title('scene'), plt.axis('off')
            
        result['p'].append(round(p, 2))
        result['s'].append(round(s, 2))          
            
        # print('no.{:2d}\t PSNR:{:.2f}\t SSIM:{:.2f}\t census error:{:.2f}'.format(i, p, s, ce))
        # print('no.{:2d}\t PSNR:{:.2f}\t SSIM:{:.2f}\t'.format(i, p, s))
        if save:
            with open(save_file, 'a') as f:
                f.write('no.{:2d}\t PSNR:{:.2f}\t SSIM:{:.2f}\t\n'.format(i, p, s))
    return result

In [130]:
def coords_grid(batch, ht, wd):
    coords = torch.meshgrid(torch.arange(ht), torch.arange(wd))    
    coords = torch.stack(coords[::-1], dim=0).float()     

    return coords[None].repeat(batch, 1, 1, 1)
    #return np.asarray(coords)
    
def image_flow_warp(image, flow, padding_mode='zeros'):
    '''
    Input:
        image: HxWx3 numpy
        flow: HxWx2 torch.Tensor
    Output:
        outImg: HxWx3 numpy
    '''
    image = torch.from_numpy(image)
    if image.ndim == 2:
        image = image[None].permute([1,2,0])
    H, W, _ = image.shape
    coords = coords_grid(1, H, W).cuda().float().contiguous()
    flow = flow[None].repeat(1, 1, 1, 1).permute([0, 3, 1, 2]).float().contiguous()    
    grid = (flow + coords).permute([0, 2, 3, 1]).contiguous()   # (1, H, W, 2)  
    
    grid[:, :, :, 0] = (grid[:, :, :, 0] * 2 - W + 1) / (W - 1)
    grid[:, :, :, 1] = (grid[:, :, :, 1] * 2 - H + 1) / (H - 1)    
    image = image[None].permute([0, 3, 1, 2]).cuda().float()
    
    outImg = F.grid_sample(image, grid, padding_mode=padding_mode, align_corners=False)[0].cpu().numpy().transpose([1, 2, 0])
    return outImg
    
def image_forward_warp(image, flow, padding_mode='zeros'):
    '''
    Input:
        image: HxWx3 numpy
        flow: HxWx2 torch.Tensor
    Output:
        outImg: HxWx3 numpy
    '''
    forward_warp = ForwardWarp()    
    image = torch.from_numpy(image).permute([2,0,1])[None].cuda()
    # flow = torch.from_numpy(flow).permute([2,0,1])[None].cuda()
    flow = flow.permute([2,0,1])[None].cuda()
    out = forward_warp(image, flow)
    image = (out[0][0] / out[1][0] + 1e-6).cpu().permute([1,2,0]).numpy().astype(np.uint8)

    return image


In [131]:
def blend_life(scene_images, target, estimator, source=None, blend_type='scale', warp='grid_sample', use_colormap=True):    
    #H, W = 480, 640    
    blends = []
    masks = []
    blends_colormap = []
    for scene in scene_images:        
        #ori_H, ori_W = scene.shape[:2]
        #scene = cv2.resize(scene, (W, H))
        target = cv2.resize(target, (scene.shape[1], scene.shape[0]))
        if './Twins-new' in sys.path:
            flow = estimator.estimate(target, scene)           
        else:
            flow = estimator.estimate(scene, target)           
        
        if use_colormap:
            colormap = get_colormap(flow, scene.shape[0], scene.shape[1])
            #cv2.imwrite('./colormap.jpg', cv2.resize((colormap[:,:,::-1]+256)/2, (640,480)))

        mask = None
        if warp == 'grid_sample':
            if './Twins-new' in sys.path:
                out = image_forward_warp(target, flow[0].permute([1,2,0])) 
                mask_origin = (np.ones(shape=(target.shape[0], target.shape[1], 1)) * 255).astype(np.uint8)
                mask_origin = image_forward_warp(mask_origin, flow[0].permute([1,2,0]),padding_mode='zeros')
                mask = (255 - mask_origin).astype(np.float64) / 255.0 
                # mask_origin = np.ones(shape=(target.shape[0], target.shape[1], 1)).astype(np.float64) 
                # mask_origin = image_forward_warp(mask_origin, flow[0].permute([1,2,0]),padding_mode='zeros')
                # mask = (1 - mask_origin)
                if use_colormap:
                    out_colormap =  image_forward_warp(colormap, flow[0].permute([1,2,0])) 
                    out_colormap = ((out_colormap + 256) / 2)
                    mask_colormap = np.ones_like(colormap).astype(np.float32) / 2
                    mask_colormap = image_forward_warp(mask_colormap, flow[0].permute([1,2,0]))
                    mask_colormap = (1 - mask_colormap)                    
            else:
                out = image_flow_warp(target, flow[0].permute([1,2,0]))  
                mask_origin = np.ones(shape=(target.shape[0], target.shape[1], 1)).astype(np.float64) 
                mask_origin = image_flow_warp(mask_origin, flow[0].permute([1,2,0]),padding_mode='zeros')
                mask = (1 - mask_origin)
                if use_colormap:
                    out_colormap =  image_flow_warp(colormap, flow[0].permute([1,2,0])) 
                    out_colormap = ((out_colormap + 256) / 2)
                    # cv2.imwrite('./out_colormap.jpg', out_colormap)   
                    mask_colormap = np.ones_like(colormap).astype(np.float32) / 2
                    mask_colormap = image_flow_warp(mask_colormap, flow[0].permute([1,2,0]))
                    mask_colormap = (1 - mask_colormap) 

            # plt.figure()
            # plt.subplot(141), plt.imshow(out), plt.title('out')
            # plt.subplot(142), plt.imshow(mask), plt.title('mask')
            # plt.subplot(143), plt.imshow(scene), plt.title('scene')
            # plt.subplot(144), plt.imshow((scene*(1-mask)).astype(np.uint8)), plt.title('scene with mask')
                      
        elif warp == 'homography':
        # RANSAC homography
            flow = flow[0].permute([1,2,0])
            image = target
            img = image
            sce = scene
            image = torch.from_numpy(image)
            if image.ndim == 2:
                image = image[None].permute([1,2,0])
            H, W, _ = image.shape
            coords = coords_grid(1, H, W).cuda().float().contiguous()
            flow = flow[None].repeat(1, 1, 1, 1).permute([0, 3, 1, 2]).float().contiguous()    
            grid = (flow + coords).permute([0, 2, 3, 1]).contiguous()   # (1, H, W, 2)
            grid = grid[0].cpu()
            src_pts = []
            dst_pts = []
            for y in range(H):
                for x in range(W):
                    if grid[y,x,0]>=0 and grid[y,x,0]<W and grid[y,x,1]>=0 and grid[y,x,1]<H:
                        src_pts.append((grid[y,x,0], grid[y,x,1]))
                        dst_pts.append((x, y))                        
            src_pts = np.float32(src_pts)
            dst_pts = np.float32(dst_pts)
            '''
            num = len(src_pts)
            for idx in range(num):
                print(src_pts[idx], dst_pts[idx])
                cv2.circle(img, (int(src_pts[idx][0]), int(src_pts[idx][1])), 1, (255, 0, 0))
                cv2.circle(sce, (int(dst_pts[idx][0]), int(dst_pts[idx][1])), 1, (255, 0, 0))
            plt.figure(facecolor='white')
            plt.subplot(1,2,1), plt.imshow(img)
            plt.subplot(1,2,2), plt.imshow(sce)
            plt.show()
            '''
            M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
            if M is None:
                blends.append(None)
                continue
            out = cv2.warpPerspective(target, M, (scene.shape[1], scene.shape[0]))        

        blend_i, mask_i = blend(out, source, scene, blend_type, mask=mask)        
        blends.append(blend_i)
        masks.append(mask_i)
        if use_colormap:            
            blend_colormap = (out_colormap * (1 - mask_colormap) + scene * mask_colormap).astype(np.uint8) 
            blends_colormap.append(blend_colormap)
    return blends, masks, blends_colormap

In [132]:
def blend_SPSG(scene_images, target, matching=None, source=None, blend_type='scale', save_root=None, use_colormap=True):
    blends = []
    masks = []
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    for idx, scene in enumerate(scene_images):
        target = cv2.resize(target, (scene.shape[1], scene.shape[0]))
        target_gray = cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)
        scene_gray = cv2.cvtColor(scene, cv2.COLOR_BGR2GRAY)
        inp0 = frame2tensor(target_gray, device=device)
        inp1 = frame2tensor(scene_gray, device=device)        
        pred = matching({'image0': inp0, 'image1': inp1})
        pred = {k: v[0].cpu().numpy() for k, v in pred.items()}
        kpts0, kpts1 = pred['keypoints0'], pred['keypoints1']
        matches, conf = pred['matches0'], pred['matching_scores0']

        valid = matches > -1          
        mkpts0 = kpts0[valid]
        mkpts1 = kpts1[matches[valid]] 
        mconf = conf[valid]
        
        # plot_image_pair([target, scene])        
        # plot_keypoints(kpts0, kpts1, color='k', ps=4)
        # plot_keypoints(kpts0, kpts1, color='w', ps=2)
        # plot_matches(mkpts0, mkpts1, color='r')
        # plt.show()
        # plt.savefig(os.path.join(save_root, 'SPSG_match_'+str(idx)+'.jpg'), bbox_inches='tight', pad_inches=0)
        
        valid = mconf > 0.5
        mkpts0 = mkpts0[valid]
        mkpts1 = mkpts1[valid]
        
        if np.count_nonzero(valid) < 4:
            blends.append(None)
            masks.append(None)
            continue
        
        src_pts = np.float32(mkpts0).reshape(-1,1,2)
        dst_pts = np.float32(mkpts1).reshape(-1,1,2)
        M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        if M is None:
            blends.append(None)
            masks.append(None)
            continue
        out = cv2.warpPerspective(target, M, (scene.shape[1], scene.shape[0]))        

        blend_i, mask_i = blend(out, source, scene, blend_type)
        #blend_i = cv2.resize(blend_i, (ori_W, ori_H))
        blends.append(blend_i)
        masks.append(mask_i)
        
        
    return blends, masks

In [133]:
def eval(id, 
         scene_images, 
         target,           
         save_root, 
         blend_method='life',
         blend_type='scale', 
         estimator=None, 
         flow_estimator=None, 
         estimate_uncertainty=None, 
         matching=None,         
         source=None,
         detector='SIFT',
         with_mask = False,
         use_colormap=True,
         draw = True,
         save = True,
         warp = 'grid_sample',
         heatmap=False):   
    '''
    Args:
        @id: (int) target image id, for logging
        @scene_images: (list) test scene images
        @target: (array) target image
        @save_root: (string) 
        @blend_method: (string, 'life') ['life', 'homography', 'pdc']
        @blend_type: (string, 'scale') ['scale', 'light', 'viewpoint']
        @estimator: model to estimate when blend_method=='life'
        @flow_estimator: model to estimate when blend_method=='pdc'
        @estimate_uncertainty: check result uncertainty when blend_method=='pdc'
        @matching: SPSG
        @coarseModel & network: RANSAC-Flow
        @source: source image to calculate metrics when blend_type=='lighting'
        @detector: (string, 'SIFT) ['SIFT', 'ORB'] detector used in homography
        @with_mask: (bool) calculate metrics with or without mask
        @draw: (bool) show results or not
        @save: (bool) save results or not
        @warp: (string,'grid_sample') ['grid_sample', 'homography'] warp method for LIFE model
    '''
    if blend_method in ['life', 'raft', 'biraft', 'ms', 'twins', 'occ', 'twins-twostage', 'twins-onestage']:
        if estimator is None:
            raise ValueError('estimator not set')        
        out, mask, out_colormap = blend_life(scene_images, target, estimator=estimator, source=source, blend_type=blend_type, warp=warp, use_colormap=use_colormap)
    elif blend_method == 'homography':        
        out, mask = blend_homography(scene_images, target, source=source, blend_type=blend_type, detector=detector, use_colormap=use_colormap)
    elif blend_method == 'pdc':
        if flow_estimator is None:
            raise ValueError('estimator not set')
        out, mask = blend_pdc(scene_images, target, flow_estimator=flow_estimator, estimate_uncertainty=estimate_uncertainty, source=source, blend_type=blend_type, use_colormap=use_colormap)
    elif blend_method == 'SPSG':
        save_root = os.path.join(save_root, str(id).zfill(4))
        if save and not os.path.exists(save_root):
            os.makedirs(save_root)
        out, mask = blend_SPSG(scene_images, target, matching=matching, source=source, blend_type=blend_type, save_root=save_root, use_colormap=use_colormap)
    else:
        raise ValueError('blend method has not been implemented')

        
    if blend_type == 'light':
        result = metrics(id, out, mask, save_root=save_root, source=source, blend_type=blend_type, with_mask=with_mask, heatmap=heatmap, save=save)
    else:
        result = metrics(id, out, mask, save_root=save_root, scene_images =scene_images, blend_type=blend_type, with_mask=with_mask, heatmap=heatmap, save=save)
        
    if draw or save: 
        # save_root = os.path.join(save_root, str(id).zfill(4))
        # if save and not os.path.exists(save_root):
        #     os.makedirs(save_root)
        for i in range(len(scene_images)): 
            if out[i] is None:
                continue
            title = 'PSNR: '+str(result['p'][i])+' SSIM: '+str(result['s'][i])
            # plt.figure(figsize=(10,15), facecolor='white')      
                        
            # plt.subplot(2, 2, 1)
            # plt.imshow(target), plt.axis('off')# , plt.title('target '+str(id).zfill(4))        
            
            if not with_mask:
                if blend_type == 'viewpoint':
                    scene_images[i] = resize_viewpoint(scene_images[i])
                    out[i] = resize_viewpoint(out[i])
                    if use_colormap:
                        out_colormap[i] = resize_viewpoint(out_colormap[i])                    
                elif blend_type == 'light':
                    scene_images[i] = resize_lighting(scene_images[i])
                    out[i] = resize_lighting(out[i])
                    if use_colormap:
                        out_colormap[i] = resize_lighting(out_colormap[i])
            if draw: 
                plt.figure(facecolor='white')
                plt.subplot(1, 2, 1)
                plt.imshow(scene_images[i]), plt.axis('off')# , plt.title('scene '+str(i).zfill(4))        
                plt.subplot(1, 2, 2) 
                plt.imshow(out[i]), plt.axis('off')#, plt.title(title)
                # plt.subplot(1, 2, 2) 
                # if with_mask:
                #     plt.imshow(out[i]*(1-mask[i])), plt.axis('off') #, plt.title(title)  
                # else:                    
                #     plt.imshow(out[i]), plt.axis('off')                
              
            if save: 
                # plt.savefig(os.path.join(save_root,str(i)+'_'+title+'.png'), dpi=200, bbox_inches='tight')            
                io.imsave(os.path.join(save_root, str(i)+'_out.png'), out[i].astype(np.uint8))                
                io.imsave(os.path.join(save_root, str(i)+'_scene.png'), scene_images[i].astype(np.uint8))
                if use_colormap:
                    io.imsave(os.path.join(save_root, str(i)+'_out_colormap.png'), out_colormap[i])
                if heatmap:
                    io.imsave(os.path.join(save_root, str(i)+'_psnr_heatmap.png'), resize_lighting(result['psnr_heatmap'][i]))
                    io.imsave(os.path.join(save_root, str(i)+'_ssim_heatmap.png'), resize_lighting(result['ssim_heatmap'][i]))
            plt.show()
            
    return (out, result)

def resize_viewpoint(img):
    if img.shape[0] > img.shape[1]:
        resize = cv2.flip(cv2.transpose(img), 0)
        return resize
    else:
        return img

def resize_lighting(img):
    if img.shape[0] > img.shape[1]:        
        region = img[140:500, 0:480]
        resize = cv2.resize(region, (640, 480))
        return resize 
    else:
        return img

In [134]:
def run(root = './assets/',
        blend_type = 'scale',
        blend_method = 'life', 
        img_num = 5, 
        start_img_id = 0,
        start_scene_id = 0,
        scn_num = 10,
        source_id = -1,
        H = 480,
        W = 640,
        with_mask = False,
        use_colormap = True,
        draw = True,
        save = True,
        save_root="",
        detector="SIFT",
        multisample=True,
        folder="",
        warp='grid_sample',
        heatmap=False):
    '''
    Args:
        @root: (string, './assets/') image data root
        @blend_type: (string, 'scale) ['scale', 'light', 'viewpoint', 'deformation', 'occlusion']
        @blend_method: (string, 'life') ['homography', 'pdc', 'life', 'raft', 'biraft', 'ms', 'occ', 'twins', 'twins-twostage', 'twins-onestage']
        @img_num: (int, 5) number of test images
        @start_img_id: (int, 0) test from image with id=start_img_id
        @start_scene_id: (int, 0) test from scene image with id=start_scene_id
        @scn_num: (int, 10) number of scene images per test image
        @source_id: if blend_type=='light', source id should be specified for metrics calculation
        @(W, H): resize images
        @with_mask: caculate PSNR/SSIM with or without mask
        @draw: (bool)
        @save: (bool)
        @detector: (string, 'SIFT') ['SIFT', 'ORB']
        @multisample: (bool, True) if False(not one of blend_type), specify scene images folder
        @warp: (string, 'grid_sample') ['grid_sample', 'homography'] warp method for LIFE model
    Ouput:
        folder '{blend_method}_output_{time}' with output images and output.txt(metrics) will be created under folder {blend_type}
    '''
    # path setting
    if blend_type == 'light':
        if source_id == -1:
            raise FileExistsError('no source image id specified.')    
    if multisample:
        scene_root = os.path.join(root, blend_type)
    else:
        if folder=="":
            raise FileNotFoundError("input folder name")
        scene_root = os.path.join(root, folder) 
    # time = datetime.datetime.now()
    # suffix = datetime.datetime.strftime(time, '%m%d%H%M')
    # if not with_mask:
    #     save_root = os.path.join(scene_root, blend_method+'_output_'+suffix)
    # else:
    #     save_root = os.path.join(scene_root, blend_method+'_output_mask_'+suffix)
    if save and not os.path.exists(save_root):
        os.makedirs(save_root)
        print('save image to '+save_root)

    # data loading
    if H > W:
        H, W = W, H #keep H < W
    scene_images = []
    target_images = []    
    for id in range(img_num):
        id = id + start_img_id
        scene_image = []
        for i in range(start_scene_id, scn_num):            
            scene_path = os.path.join(scene_root, str(id).zfill(4) + "_" + blend_type + "_"+str(i)+".jpg")            
            scene = io.imread(scene_path)
            if scene.shape[0] > scene.shape[1]: # H > W
                scene = cv2.resize(scene, (H, W))
            else: # H < W
                scene = cv2.resize(scene, (W, H))
            scene_image.append(scene)
        scene_images.append(scene_image)


        target_path = os.path.join(root, str(id).zfill(4) + ".jpg")
        target_image = io.imread(target_path)
        if target_image.shape[0] > target_image.shape[1]: # H > W
            target_image = cv2.resize(target_image, (H, W))
        else:
            target_image = cv2.resize(target_image, (W, H))        
        target_images.append(target_image)

    total = len(scene_images)*len(scene_images[0])
    print('input {}x{} scene images'.format(len(scene_images), len(scene_images[0])))    
    print('input {} target images'.format(len(target_images)))
        
    # show_input_images(scene_images, target_images)

    # model loading
    print('blend method: ', blend_method)    
        
    if blend_method == 'pdc':
        args = easydic({
            'model': 'PDCNet',
            'pre_trained_model_type': 'megadepth',
            'path_to_pre_trained_models': '/home/weihong/work/Optical_Flow/DenseMatching/pre_trained_models',
            'local_optim_iter': 7,
            'optim_iter': 3,
            'network_type': None
        })
        flow_estimator, estimate_uncertainty = select_model(
            args.model, 
            args.pre_trained_model_type, 
            args,
            args.optim_iter, 
            args.local_optim_iter,
            path_to_pre_trained_models=args.path_to_pre_trained_models)

    elif blend_method == 'SPSG':
        opt = easydic({
            'nms_radius': 4,
            'keypoint_threshold': 0.005,
            'max_keypoints': 1024,
            'superglue': 'indoor',
            'sinkhorn_iterations': 20,
            'match_threshold': 0.2,
        })
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print('Running inference on device \"{}\"'.format(device))
        config = {
            'superpoint': {
                'nms_radius': opt.nms_radius,
                'keypoint_threshold': opt.keypoint_threshold,
                'max_keypoints': opt.max_keypoints
            },
            'superglue': {
                'weights': opt.superglue,
                'sinkhorn_iterations': opt.sinkhorn_iterations,
                'match_threshold': opt.match_threshold,
            }
        }
        matching = Matching(config).eval().to(device)
    elif blend_method == 'homography':
        print('detector: '+ detector)
    else:
        if blend_method == 'life':
            args = get_life_args()  
        elif 'twins' in blend_method:
            args = get_twins_args()
            args.fnet = 'twins'
            if 'twostage' in blend_method:
                args.model = './model/twostage.pth'
                args.twoscale = True
            elif 'onestage' in blend_method:
                args.model = './model/twins-onestage.pth'
                args.twoscale = False
                args.dim_corr_all = 192
        else:
            args = get_raft_args()
            if blend_method == 'biraft':
                args.model = './model/biRAFT_latest.pth'
            elif blend_method == 'ms':
                args.model = './model/MultiScale_latest.pth'
            elif blend_method=='occ':
                args.model = "./model/RAFT_twins_bs12_ep20.pth"  
                args.fnet = 'twins'              
        #for k, v in vars(args).items():
        #    print(k,'=',v)
        #estimator = Flow_estimator(args, method=blend_method)
        estimator = Flow_estimator(args, blend_method)
        print('model: ', args.model)
        print('warp method: ', warp)

            
    ID = []
    #out_images = []
    #results = []
    p = []
    s = []
    fail = 0
    for id in range(img_num):
        scene = scene_images[id]
        target = target_images[id]        
        source = scene[source_id]  
        if blend_method in ['life', 'raft', 'biraft', 'ms', 'twins', 'occ', 'twins-twostage', 'twins-onestage']:
            out, result = eval(id, scene, target, blend_method=blend_method, save_root=save_root, source=source, blend_type=blend_type, estimator=estimator, with_mask=with_mask, use_colormap=use_colormap, heatmap=heatmap, draw=draw, save=save, warp=warp)
        elif blend_method == 'pdc':
            out, result = eval(id, scene, target, blend_method=blend_method, save_root=save_root, source=source, blend_type=blend_type, flow_estimator=flow_estimator, estimate_uncertainty=estimate_uncertainty, with_mask=with_mask, use_colormap=use_colormap, heatmap=heatmap, draw=draw, save=save)
        elif blend_method == 'homography':
            out, result = eval(id, scene, target, blend_method=blend_method, save_root=save_root, source=source, blend_type=blend_type, with_mask=with_mask, detector=detector, use_colormap=use_colormap, heatmap=heatmap, draw=draw, save=save)
        elif blend_method == 'SPSG':
            out, result = eval(id, scene, target, blend_method=blend_method, save_root=save_root, source=source, blend_type=blend_type, matching=matching, with_mask=with_mask, use_colormap=use_colormap, heatmap=heatmap, draw=draw, save=save)
        else:
            raise ValueError('blend method not inplemented')
        #out_images.append(out)
        #results.append(result)
        #ID.append(str(id).zfill(4))
        
        # Include Failures
        p.append(result['p'])
        s.append(result['s'])

        # Exclude Failures
        # p.append([result['p'][i] for i in range(len(result['p'])) if result['p'][i]!=-1])
        # s.append([result['s'][i] for i in range(len(result['s'])) if result['s'][i]!=-1])
        fail = fail + result['fail']
    
    # p_array = np.asarray(p).flatten()
    # s_array = np.asarray(s).flatten()
    p_array = np.asarray([item for sub in p for item in sub])    
    s_array = np.asarray([item for sub in s for item in sub])
    # print(p_array)
    # print(s_array)
    # mask = ~np.isnan(p_array)
    # p_array = p_array[mask]
    # s_array = s_array[mask]
    
    print(blend_method+'\t PSNR: {:.2f}/{:.2f}\t SSIM: {:.2f}/{:.2f}\t fail: {:.2f}%({}/{})'
        .format(np.mean(p_array), 
                np.median(p_array),
                np.mean(s_array), 
                np.median(s_array),
                fail*1.0/total*100,
                fail, total))
    
    #draw_result(ID, out_images, results)

In [135]:
''' 数据集格式
|-deformation (scene images)
    |-0000_deformation_0.jpg
    |-0000_deformation_1.jpg
    |-...
    |-0001_deformation_0.jpg
    |-...
|-light
|-occlusion
|-scale
|-viewpoint
|-0000.jpg (target image)
|-0001.jpg
|-...
'''
# life / multiscale / twins 记得改模型路径！！！
# !!! pip install kornia == 0.6.3
types = ['viewpoint', 'light', 'deformation', 'scale', 'occlusion']
blend_type = 'viewpoint'
blend_method = 'SPSG'
warp = 'grid_sample'
with_mask = True
draw = False 
save = False
use_colormap = False # mask output with colormap
heatmap = False # output metrics heatmap
img_num = 10
scn_num = 10
start_img_id = 0
start_scene_id = 0
source_id = 1

 
time = datetime.datetime.now()
suffix = datetime.datetime.strftime(time, '%m%d%H%M')        
save_root = os.path.join('./assets/output', blend_method+'_output_'+suffix+'_'+str(start_img_id))

for blend_type in types:
    print('blend type: '+blend_type)
    save_dir = os.path.join(save_root, blend_type)
    if save:
        os.makedirs(save_dir, exist_ok=True)
    run(blend_type=blend_type, 
        blend_method=blend_method, 
        warp=warp, 
        img_num=img_num, 
        scn_num=scn_num, 
        source_id=source_id, 
        with_mask=with_mask, 
        use_colormap=use_colormap,
        multisample=True, 
        folder="minions", 
        start_img_id=start_img_id, 
        start_scene_id = start_scene_id, 
        draw=draw, 
        save=save, 
        save_root=save_dir,
        detector="SIFT",
        heatmap=heatmap)

# sift / pdc
# run(blend_type='occlusion', blend_method='homography', warp='grid_sample', img_num=1, scn_num=10, source_id=1, with_mask=True, multisample=True, folder="minions", start_img_id=0, start_scene_id = 0, draw=False, save=False, detector="SIFT")

# SPSG
# run(blend_type='deformation', blend_method='SPSG', warp='grid_sample', img_num=1, scn_num=10, source_id=1, with_mask=True, multisample=True, folder="minions", start_img_id=0, start_scene_id = 0, draw=False, save=False, detector="SIFT")

# LIFE - life->pretrain.pth / raft->RAFT_latest.pth / biraft->biRAFT_latest.pth
# sys.path.append('./LIFE')
# sys.path.append('./LIFE/core')
# run(blend_type='viewpoint', blend_method='biraft', warp='grid_sample', img_num=10, scn_num=10, source_id=1, with_mask=True, multisample=True, folder="minions", start_img_id=0, start_scene_id = 0, draw=False, save=False, detector="SIFT")


# MultiScale - MultiScale_latest.pth
# sys.path.append('./MultiScale')
# sys.path.append('./MultiScale/core')
# run(blend_type='viewpoint', blend_method='ms', warp='grid_sample', img_num=10, scn_num=10, source_id=1, with_mask=True, multisample=True, folder="minions", start_img_id=0, start_scene_id = 0, draw=False, save=False, detector="SIFT")

# Occ - RAFT_twins_bs12_ep20.pth
# sys.path.append('./Occ')
# sys.path.append('./Occ/core')
# run(blend_type='viewpoint', blend_method='occ', warp='grid_sample', img_num=10, scn_num=10, source_id=1, with_mask=True, multisample=True, folder="minions", start_img_id=0, start_scene_id = 0, draw=False, save=False, detector="SIFT")

# Twins - best_twins_ms.pth / twins-twostage.pth / twins-onestage.pth
# sys.path.append('./Twins')
# sys.path.append('./Twins/core')
# run(blend_type='viewpoint', blend_method='twins', warp='grid_sample', img_num=10, scn_num=10, source_id=1, with_mask=True, multisample=True, folder="minions", start_img_id=0, start_scene_id = 0, draw=False, save=False, detector="SIFT")
# run(blend_type='deformation', blend_method='twins-onestage', warp='grid_sample', img_num=10, scn_num=10, source_id=1, with_mask=True, multisample=True, folder="minions", start_img_id=0, start_scene_id = 0, draw=False, save=False, detector="SIFT")



blend type: viewpoint
input 10x10 scene images
input 10 target images
blend method:  SPSG
Running inference on device "cuda"
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
SPSG	 PSNR: 11.77/13.89	 SSIM: 0.34/0.63	 fail: 19.00%(19/100)
blend type: light
input 10x10 scene images
input 10 target images
blend method:  SPSG
Running inference on device "cuda"
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
SPSG	 PSNR: 11.30/13.52	 SSIM: 0.34/0.67	 fail: 21.00%(21/100)
blend type: deformation
input 10x10 scene images
input 10 target images
blend method:  SPSG
Running inference on device "cuda"
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
SPSG	 PSNR: 10.00/10.93	 SSIM: 0.17/0.29	 fail: 12.00%(12/100)
blend type: scale
input 10x10 scene images
input 10 target images
blend method:  SPSG
Running inference on device "cuda"
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
SPSG	 PSNR: 2.46/0.00	 SSIM: -0.70/-1.00	 fail: 78.00